In [1]:
#!pip install numpy pandas xpress PySCIPopt matplotlib networkx 

In [3]:
import numpy as np 
import pandas as pd 
from pyscipopt import Model, quicksum
from read_data import process_all_instances

In [15]:
all_instance_data = process_all_instances("./training_data")

Processing instance: 1d7c5373-e121-4d33-8dac-32f8801f7b6a
Processing instance: 3540d0d3-4667-444b-8700-1612f951c049
Processing instance: 4f90ea3a-aee4-4a3e-a862-c073e5390801
Processing instance: 6ed67a32-bfb7-4207-8873-6e0740467db5
Processing instance: 99a03a35-ab56-4bd8-8b1d-42a7f51969c8
Processing instance: 9ca9e06e-e1ce-4a05-b847-c1b0ecdd5e29
Processing instance: 16a60115-053d-4102-b189-1d2e1b2164a5
Processing instance: 49ecb4a1-ebdf-4826-836b-e02698de07fa
Processing instance: 4845b3e3-1ff2-44c2-9c0c-50874d4fab5b
Processing instance: a1e08921-ab7f-418d-b140-7034983c387a
Processing instance: 8c0c8ce3-bc0c-4422-9370-d8dcc45bdf7f
Processing instance: 9c1ad818-ec6f-4fd0-a1cb-485b6da0c22c
Processing instance: 5d8f0aa5-8639-476b-a314-f2bab5232403
Processing instance: b5d9b099-6e8a-4c2b-a11c-8db444257ab1
Processing instance: 1adef166-1111-45fd-b722-0f817c7fa055
Processing instance: 13346e71-0380-4feb-945a-2688f3dbd41b
Processing instance: 59d81edb-ec52-4bd4-9c4b-d771a234e88b
Processing ins

In [148]:
keys= list(all_instance_data[0].keys())

instance_id=14
instance = all_instance_data[instance_id]

name= instance[keys[0]]

couriers= instance[keys[1]]

orders= instance[keys[2]]

travel_time= instance[keys[3]]

print(name)

1adef166-1111-45fd-b722-0f817c7fa055


In [149]:
couriers[0]

Courier(ID=1, Location=1, Capacity=100)

In [150]:
orders

[Delivery(ID=3, Capacity=100, Pickup Loc=3, Time Window Start=5, Pickup Stacking Id=42187, Dropoff Loc=4),
 Delivery(ID=4, Capacity=35, Pickup Loc=5, Time Window Start=27, Pickup Stacking Id=12603, Dropoff Loc=6)]

In [151]:
travel_time

[['Locations', '1', '2', '3', '4', '5', '6'],
 [1, 0, 10, 12, 8, 7, 24],
 [2, 10, 0, 10, 17, 17, 32],
 [3, 12, 10, 0, 20, 19, 34],
 [4, 8, 17, 20, 0, 6, 14],
 [5, 7, 17, 19, 6, 0, 16],
 [6, 24, 32, 34, 14, 16, 0]]

In [176]:
COURIERS = [i for i in range(len(couriers))]

# determine the depos and Capacities 
DEPOS= []
CAPACITIES = []
for i in COURIERS:
      DEPOS.append(couriers[i].location)
      CAPACITIES.append(couriers[i].capacity)
MAXCAP= max(CAPACITIES)
# Determine job data 
ORDERS = [i for i in range(len(orders))]
PICKUPS = []
DROPOFFS = []
SIZES = {} 
TWSTART = []
for i in ORDERS:
      PICKUPS.append(orders[i].pickup_loc)
      DROPOFFS.append(orders[i].dropoff_loc)
      SIZES[orders[i].pickup_loc]= orders[i].capacity 
      SIZES[orders[i].dropoff_loc] =  - orders[i].capacity 
      TWSTART.append(orders[i].time_window_start)
      
      
VERTICES= list(set(DEPOS + PICKUPS + DROPOFFS))

A_1 = [(u,v) for u,v in zip(DEPOS,PICKUPS)]
A_2 = [(u,v) for u,v in zip(PICKUPS,DROPOFFS) if u!=v]
A_3 = [(u,v) for u,v in zip(DROPOFFS,DEPOS)]
A_4 = [(u,u) for u in DEPOS]


DISTANCES= np.array(travel_time)[1:,1:].astype(np.int16)

# defined as a sum all of travel time - just an arbitrary large number 
T = np.sum(DISTANCES.flatten())
print(T)

if np.count_nonzero(DISTANCES)!= DISTANCES.shape[0]**2 - DISTANCES.shape[0]:
      print('Not fully connected matrix')

492


In [177]:
print(VERTICES)

[1, 2, 3, 4, 5, 6]


In [178]:
DISTANCES

array([[ 0, 10, 12,  8,  7, 24],
       [10,  0, 10, 17, 17, 32],
       [12, 10,  0, 20, 19, 34],
       [ 8, 17, 20,  0,  6, 14],
       [ 7, 17, 19,  6,  0, 16],
       [24, 32, 34, 14, 16,  0]], dtype=int16)

In [180]:
# let us model the problem with scip

from pyscipopt import Model, quicksum
import itertools

model = Model()

t = {}
for i in VERTICES:
        t[i] = model.addVar(vtype="C", name=f't[{i}]')

q= {}
for i in VERTICES:
        q[i] = model.addVar(vtype="C", name=f'q[{i}]', ub=MAXCAP)

x={}
for i in VERTICES:
    for j in VERTICES:
           for k in COURIERS:
                x[i,j,k]= model.addVar(vtype='B', name=f'x[{i},{j},{k}]')

# 2
# every vertex should be visited only ones 
for v in VERTICES:
    model.addCons(quicksum(x[i,v,k] for i in VERTICES for k in COURIERS) == 1)

# 3 
# the same vehicle that visits a vertex does also leave it 
for v in VERTICES :
      for k in COURIERS:
            model.addCons(quicksum(x[v,u,k] for u in VERTICES)- quicksum(x[u,v,k] for u in VERTICES) == 0)

# 4 
# guarantee that the same vehicle that visits a
# pickup vertex does also visit the corresponding delivery vertex

for o in ORDERS:
        for k in COURIERS:
                model.addCons(quicksum(x[u,PICKUPS[o],k] for u in VERTICES) - quicksum(x[u,DROPOFFS[o],k] for u in VERTICES) == 0)

# 5 
# only the vehicle corresponding to its depot can leave it 
for k in COURIERS:
      model.addCons(quicksum(x[DEPOS[k],v,k] for v in VERTICES) ==1) 

# 6
# ensure that the time variables for the vertices are correctly updated 
# A1 from depos to pickup
# A2 form pickup to dropoff (u!=v)

A_1 = [(u,v) for u in DEPOS for v in PICKUPS]
A_2 = [(u,v) for u in PICKUPS for v in DROPOFFS if u!=v]

for k in COURIERS:
      for arc in list(set(A_1 + A_2)):
            u,v = arc[0],arc[1]      
            model.addCons(t[u] + (DISTANCES[u-1,v-1] + T)*x[u,v,k] <= t[v] + T)
            

# 7 
# ensure that the pickup is visited before the corresponding dropoff.
for o in ORDERS:
      model.addCons(t[DROPOFFS[o]] - t[PICKUPS[o]] >= DISTANCES[DROPOFFS[o]-1,PICKUPS[o]-1])

# 8 

# A3 from dropoff to depos 
# A4 loops on starts 

for k in COURIERS:
      for arc in list(set(A_1 + A_2)):
            u,v = arc[0],arc[1]
            model.addCons(q[u] + (SIZES[v] + MAXCAP)*x[u,v,k] <= q[v] + MAXCAP)

# 9 
# load at the depots is already fixed
for k in COURIERS:
      model.addCons(q[DEPOS[k]]== CAPACITIES[k])

# 10 
# t is representing the time when a vertex is visited by a vehicle. As we can see, this
# variable has to respect the bounds of the respective time window.

for o in ORDERS:
      model.addCons(t[PICKUPS[o]]>= TWSTART[o])
      
model.setObjective(quicksum(t[v] for v in DROPOFFS))
      

model.optimize() 
if model.getNSols() == 0:
    print("No feasible solution found.")
else:
    print(f'Objective value={model.getObjVal()}')
    print(f'Solution: x={model.getBestSol()}')       


presolving:
Objective value=68.0
Solution: x={'x[1,1,0]': 0.0, 'x[1,1,1]': 0.0, 'x[1,2,0]': -0.0, 'x[1,2,1]': -0.0, 'x[1,3,0]': -0.0, 'x[1,3,1]': -0.0, 'x[1,4,0]': 0.0, 'x[1,4,1]': -0.0, 'x[1,5,0]': -0.0, 'x[1,5,1]': -0.0, 'x[1,6,0]': 1.0, 'x[1,6,1]': -0.0, 'x[2,1,0]': -0.0, 'x[2,1,1]': -0.0, 'x[2,2,0]': 0.0, 'x[2,2,1]': 0.0, 'x[2,3,0]': -0.0, 'x[2,3,1]': -0.0, 'x[2,4,0]': -0.0, 'x[2,4,1]': 1.0, 'x[2,5,0]': -0.0, 'x[2,5,1]': -0.0, 'x[2,6,0]': -0.0, 'x[2,6,1]': 0.0, 'x[3,1,0]': 0.0, 'x[3,1,1]': -0.0, 'x[3,2,0]': -0.0, 'x[3,2,1]': 0.0, 'x[3,3,0]': 0.0, 'x[3,3,1]': 1.0, 'x[3,4,0]': 0.0, 'x[3,4,1]': 0.0, 'x[3,5,0]': 0.0, 'x[3,5,1]': 0.0, 'x[3,6,0]': 0.0, 'x[3,6,1]': 0.0, 'x[4,1,0]': 0.0, 'x[4,1,1]': -0.0, 'x[4,2,0]': -0.0, 'x[4,2,1]': 1.0, 'x[4,3,0]': 0.0, 'x[4,3,1]': 0.0, 'x[4,4,0]': 0.0, 'x[4,4,1]': 0.0, 'x[4,5,0]': 0.0, 'x[4,5,1]': 0.0, 'x[4,6,0]': 0.0, 'x[4,6,1]': 0.0, 'x[5,1,0]': 0.0, 'x[5,1,1]': -0.0, 'x[5,2,0]': -0.0, 'x[5,2,1]': 0.0, 'x[5,3,0]': 0.0, 'x[5,3,1]': 0.0, 'x[5,4,0]': 0.

In [162]:
sol = model.getBestSol()



In [165]:
sol

{'x[1,1,0]': 0.0, 'x[1,1,1]': 0.0, 'x[1,2,0]': -0.0, 'x[1,2,1]': -0.0, 'x[1,3,0]': -0.0, 'x[1,3,1]': -0.0, 'x[1,4,0]': 0.0, 'x[1,4,1]': -0.0, 'x[1,5,0]': -0.0, 'x[1,5,1]': -0.0, 'x[1,6,0]': 1.0, 'x[1,6,1]': -0.0, 'x[2,1,0]': -0.0, 'x[2,1,1]': -0.0, 'x[2,2,0]': 0.0, 'x[2,2,1]': 0.0, 'x[2,3,0]': -0.0, 'x[2,3,1]': -0.0, 'x[2,4,0]': -0.0, 'x[2,4,1]': 1.0, 'x[2,5,0]': -0.0, 'x[2,5,1]': -0.0, 'x[2,6,0]': -0.0, 'x[2,6,1]': 0.0, 'x[3,1,0]': 0.0, 'x[3,1,1]': -0.0, 'x[3,2,0]': -0.0, 'x[3,2,1]': 0.0, 'x[3,3,0]': 0.0, 'x[3,3,1]': 1.0, 'x[3,4,0]': 0.0, 'x[3,4,1]': 0.0, 'x[3,5,0]': 0.0, 'x[3,5,1]': 0.0, 'x[3,6,0]': 0.0, 'x[3,6,1]': 0.0, 'x[4,1,0]': 0.0, 'x[4,1,1]': -0.0, 'x[4,2,0]': -0.0, 'x[4,2,1]': 1.0, 'x[4,3,0]': 0.0, 'x[4,3,1]': 0.0, 'x[4,4,0]': 0.0, 'x[4,4,1]': 0.0, 'x[4,5,0]': 0.0, 'x[4,5,1]': 0.0, 'x[4,6,0]': 0.0, 'x[4,6,1]': 0.0, 'x[5,1,0]': 0.0, 'x[5,1,1]': -0.0, 'x[5,2,0]': -0.0, 'x[5,2,1]': 0.0, 'x[5,3,0]': 0.0, 'x[5,3,1]': 0.0, 'x[5,4,0]': 0.0, 'x[5,4,1]': 0.0, 'x[5,5,0]': 1.0, 'x[5,5,1

In [146]:
list(set(A_1 + A_2))

[(3, 4), (1, 5), (5, 4), (2, 3), (5, 6), (3, 6), (2, 5), (1, 3)]

In [152]:
orders

[Delivery(ID=3, Capacity=100, Pickup Loc=3, Time Window Start=5, Pickup Stacking Id=42187, Dropoff Loc=4),
 Delivery(ID=4, Capacity=35, Pickup Loc=5, Time Window Start=27, Pickup Stacking Id=12603, Dropoff Loc=6)]

In [36]:
model = Model()
help(model.addVar)

Help on method addVar in module pyscipopt.scip:

addVar(name='', vtype='C', lb=0.0, ub=None, obj=0.0, pricedVar=False, pricedVarScore=1.0) method of pyscipopt.scip.Model instance
    Create a new variable. Default variable is non-negative and continuous.
    
    :param name: name of the variable, generic if empty (Default value = '')
    :param vtype: type of the variable: 'C' continuous, 'I' integer, 'B' binary, and 'M' implicit integer
    (see https://www.scipopt.org/doc/html/FAQ.php#implicitinteger) (Default value = 'C')
    :param lb: lower bound of the variable, use None for -infinity (Default value = 0.0)
    :param ub: upper bound of the variable, use None for +infinity (Default value = None)
    :param obj: objective value of variable (Default value = 0.0)
    :param pricedVar: is the variable a pricing candidate? (Default value = False)
    :param pricedVarScore: score of variable in case it is priced, the higher the better (Default value = 1.0)

